## Book to collect and store the Colors for all D1 Hockey Teams

In [24]:
## Depedencies

import pandas as pd
import numpy as np


import requests
from bs4 import BeautifulSoup

import re
import time

## Data Source: https://teamcolorcodes.com/ncaa-color-codes/

## Extract the list of Team names and their respective URLs
# List of teams is stored in div class="entry-content"

## Use a table that I found on Kaggle - Source: https://www.kaggle.com/datasets/coreymaxedon/ncaa-team-color/
path_to_table = '../Chart_and_plot_Workbooks/color_teams.csv'

# read it with pandas
colors_df = pd.read_csv(path_to_table)


# Transform the table to a wider structure
- pivot the table to a single row for each school

In [25]:
# Pivot the table again, this time correctly naming the columns based on the 'color_level'
# The approach will be to create a wide table with each color level as a column
colors_wide = colors_df.pivot(index='Team', columns='color_level', values='Hex_Code').reset_index()

# Generate column names based on the number of colors per team, which is dynamic
color_columns = ['primary_color'] + [f'color_{i}' for i in range(2, colors_wide.shape[1])]

# Rename the columns
colors_wide.columns = ['School'] + color_columns

# Display the first few rows of the transformed dataframe
colors_wide.head()




,School,primary_color,color_2,color_3,color_4,color_5,color_6,color_7
0,Abilene Christian Wildcats,#4f2170;,#ffffff;,#c5c6c8;,NaN,NaN,NaN,NaN
1,Air Force Falcons,#003087;,#8A8D8F;,NaN,NaN,NaN,NaN,NaN
2,Akron Zips,#041E42;,#A89968;,#FFFFFF;,NaN,NaN,NaN,NaN
3,Alabama A&M Bulldogs,#660000;,#ffffff;,NaN,NaN,NaN,NaN,NaN
4,Alabama Crimson Tide,#9e1b32;,#828a8f;,#ffffff;,NaN,NaN,NaN,NaN


## Get list of Division 1 Hockey Teams
- Load the csv of current season get unique team names



In [26]:
## Load csv file with school names and URLs

csv = '../../TEMP/current_season.csv'

# Load
current_season_df = pd.read_csv(csv)

# make a list of all teams in the dataset (Home Team and Away team columns)
teams = list(current_season_df['Home_Team'].unique()) + list(current_season_df['Away_Team'].unique())

# remove duplicates
teams = list(set(teams))

len(teams)

# current_season_df.columns

# List first 5 teams
teams

## Convert the list of teams to all lowercase
hockey_teams_normalized = [team.lower() for team in teams]


In [27]:

# Normalize the School names in the colors_wide dataframe and the hockey_teams list
colors_wide['School_normalized'] = colors_wide['School'].str.lower()


# Define a function to match the institution names
def match_institution(school_name, institution_list):
    # Check if any institution name is contained within the school name
    return any(institution in school_name for institution in institution_list)

# Apply the function to filter the dataframe
filtered_colors = colors_wide[colors_wide['School_normalized'].apply(lambda x: match_institution(x, hockey_teams_normalized))]

# Display the filtered dataframe
filtered_colors.head()

,School,primary_color,color_2,color_3,color_4,color_5,color_6,color_7,School_normalized
1,Air Force Falcons,#003087;,#8A8D8F;,NaN,NaN,NaN,NaN,NaN,air force falcons
10,Arizona State Sun Devils,#8C1D40;,#FFC627;,NaN,NaN,NaN,NaN,NaN,arizona state sun devils
11,Arizona Wildcats,#CC0033;,#003366;,NaN,NaN,NaN,NaN,NaN,arizona wildcats
16,Army Black Knights,#FFD530;,#D4BF91;,#B2B4B3;,#000000;,#FFFFFF;,NaN,NaN,army black knights
26,Boston College Eagles,#98002E;,#BC9B6A;,#726158;,NaN,NaN,NaN,NaN,boston college eagles


In [28]:
filtered_colors.shape


# Create a set of matched teams from the filtered dataframe
matched_teams = set(filtered_colors['School_normalized'].apply(lambda x: next((team for team in hockey_teams_normalized if team in x), None)))

# Create a set from the hockey_teams_normalized list
hockey_teams_set = set(hockey_teams_normalized)

# Find the unmatched teams by subtracting the matched set from the full set
unmatched_teams = hockey_teams_set - matched_teams

# Convert the unmatched_teams to a list and sort it for easier reading
unmatched_teams_list = sorted(list(unmatched_teams))

# Display the unmatched teams
unmatched_teams_list


['alaska',
 'alaska-anchorage',
 "american int'l",
 'arizona state',
 'augustana',
 'bemidji state',
 'bentley',
 'clarkson',
 'colorado college',
 'ferris state',
 'grand valley st.',
 'guelph',
 'lake superior',
 'lindenwood',
 'manitoba',
 'mass.-lowell',
 'massachusetts',
 'mcgill',
 'mercyhurst',
 'michigan state',
 'michigan tech',
 'minnesota state',
 'minnesota-duluth',
 'northern michigan',
 'ottawa',
 'quinnipiac',
 'rensselaer',
 'rit',
 'royal military',
 'simon fraser',
 'st. anselm',
 'st. cloud state',
 'st. lawrence',
 'st. thomas',
 'stonehill',
 'tmu',
 'toronto',
 'union',
 'us under-18',
 'vermont',
 'western michigan']

In [29]:
# ######## Quickly Booted and Blocked by the website ########

# list_url = "https://teamcolorcodes.com/ncaa-color-codes/"

# response = requests.get(list_url)
# soup = BeautifulSoup(response.text, 'html.parser')

# ## Extract the list of Team names and their respective URLs

# team_list = soup.find_all('div', class_="entry-content")[0].find_all('a')


# # for team in team_list:
# #     team_names.append(team.text)



# # Extract link and team name and store in a dataframe
# team_links = []
# team_names = []

# for team in team_list:
#     team_links.append(team['href'])
#     team_names.append(team.text)

# # Create a dataframe with team names and links
# links_df = pd.DataFrame({'team_name': team_names, 'team_link': team_links})

# # links_df.head(50)

# # Drop the fist 32 rows as they are not teams but conferences
# links_df = links_df.drop(links_df.index[0:32])

# links_df.head(50)

# # print(soup.prettify())